In [1]:
import pandas as pd
import numpy as np

In [2]:
# The path to file
file = "purchase_data.json"
file_df = pd.read_json(file, orient = "records")
file_df.head()


,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [4]:
#Total number of players
player_demographics = file_df.loc[:, ["Gender", "SN", "Age"]]
player_demographics.head()

,Gender,SN,Age
0,Male,Aelalis34,38
1,Male,Eolo46,21
2,Male,Assastnya25,34
3,Male,Pheusrical25,21
4,Male,Aela59,23


In [6]:
#Total number of players
player_demographics = player_demographics.drop_duplicates()
num_players = player_demographics.count()[0]
num_players

573

In [7]:

pd.DataFrame({"Total Players": [num_players]})

,Total Players
0,573


In [9]:
#Total number of unique items
average_item_price = file_df["Price"].mean()
total_purchase_value = file_df["Price"].sum()
purchase_count = file_df["Price"].count()
item_count = len(file_df["Item ID"].unique())

# Create DataFrame to hold results
summary_table = pd.DataFrame({"Number of Unique Items": item_count,
                               "Total Revenue": [total_purchase_value],
                               "Number of Purchases": [purchase_count],
                               "Average Purchase Price": [average_item_price]})

# Formating
summary_table = summary_table.round(2)
summary_table ["Average Price"] = summary_table["Average Purchase Price"].map("${:,.2f}".format)
summary_table ["Total Revenue"] = summary_table["Total Revenue"].map("${:,.2f}".format)
summary_table = summary_table.loc[:, ["Number of Unique Items", "Average Purchase Price", "Number of Purchases", "Total Revenue"]]

summary_table

,Number of Unique Items,Average Purchase Price,Number of Purchases,Total Revenue
0,183,2.93,780,"$2,286.33"


In [19]:
# Purchase count
gender_demographics_totals = player_demographics["Gender"].value_counts()
gender_demographics_percents = gender_demographics_totals / num_players * 100

# Rename columns
gender_demographics = pd.DataFrame({"Total Count": gender_demographics_totals,
                                    "Percentage of Players": gender_demographics_percents})

gender_demographics = gender_demographics.round(2)


gender_demographics

,Percentage of Players,Total Count
Male,81.15,465
Female,17.45,100
Other / Non-Disclosed,1.40,8


In [15]:
# Purchasing analysis
gender_purchase_total = file_df.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
gender_average = file_df.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Value")
gender_counts = file_df.groupby(["Gender"]).count()["Price"].rename("Purchase Count")


# normalized Purchasing
normalized_total = gender_purchase_total / gender_demographics["Total Count"]

# Rename columns
gender_data = pd.DataFrame({"Normalized Total": normalized_total, 
                            "Purchase Count": gender_counts, 
                            "Total Purchase Value": gender_purchase_total, 
                            "Average Purchase Value": gender_average})
gender_data

,Average Purchase Value,Normalized Total,Purchase Count,Total Purchase Value
Gender,,,,
Female,2.815515,3.829100,136,382.91
Male,2.950521,4.016516,633,1867.68
Other / Non-Disclosed,3.249091,4.467500,11,35.74


In [16]:
age_bins = [0, 9.90, 14.90, 19.90, 24.9, 29.9, 34.90, 39.90, 9999999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", ">40"]

#Age Demographics
player_demographics["Age Ranges"] = pd.cut(player_demographics["Age"], age_bins, labels=group_names)
player_demographics.head()

,Gender,SN,Age,Age Ranges
0,Male,Aelalis34,38,35-39
1,Male,Eolo46,21,20-24
2,Male,Assastnya25,34,30-34
3,Male,Pheusrical25,21,20-24
4,Male,Aela59,23,20-24


In [18]:
age_demographics_totals = player_demographics["Age Ranges"].value_counts()
age_demographics_percents = age_demographics_totals / num_players * 100

age_demographics = pd.DataFrame({"Total Count": age_demographics_totals, "Percent of Players": age_demographics_percents})
age_demographics = age_demographics.sort_index()
age_demographics

,Percent of Players,Total Count
<10,3.315881,19
10-14,4.013962,23
15-19,17.452007,100
20-24,45.200698,259
25-29,15.183246,87
30-34,8.202443,47
35-39,4.712042,27
>40,1.919721,11


In [21]:
#Top spenders
user_total = file_df.groupby(["SN"]).sum()["Price"].rename("Total Purchase Amount")
user_average = file_df.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
user_count = file_df.groupby(["SN"]).count()["Price"].rename("Purchase Count")

user_data = pd.DataFrame({"Total Purchase Amount": user_total,
                          "Average Purchase Price": user_average,
                          "Purchase Count": user_count})

# Data Frame
user_data.sort_values("Total Purchase Amount", ascending=False).head(5)

,Average Purchase Price,Purchase Count,Total Purchase Amount
SN,,,
Undirrala66,3.412000,5,17.06
Saedue76,3.390000,4,13.56
Mindimnya67,3.185000,4,12.74
Haellysu29,4.243333,3,12.73
Eoda93,3.860000,3,11.58


In [23]:
#Most popular items
user_total = file_df.groupby(["Item ID", "Item Name"]).sum()["Price"]
user_total.head()

Item ID  Item Name         
0        Splinter              1.82
1        Crucifer              9.12
2        Verdict               3.40
3        Phantomlight          1.79
4        Bloodlord's Fetish    2.28
Name: Price, dtype: float64